## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [18]:
from datetime import datetime
from scipy import stats
import os
import pytz
import quandl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

### Creating IDX Equities Time Series

In [2]:
# Location of the IDX Stocks OHLC csv files
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv'
csv_loc = '/home/nikki/documents/amibroker/exported-csv'
# csv_loc = '/Users/nikki/Documents/idx_exported_csv'

# Backup if the first one doesn't work
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv_gama'

tickers = os.listdir(csv_loc)

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for ticker in tickers:
    data_idx[ticker[:-4]] = pd.read_csv(f'{csv_loc}/{ticker}',
                                        index_col='date',
                                        parse_dates=['date'])

In [8]:
def volatility(ts, period=20):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

# Starter System

Instrument
- AUDUSD

Opening Rule
- if 16-day SMA > 64-day SMA: go long
- if 16-day SMA < 64-day SMA: go short

Position sizing
- Notional exposure = (target risk % x capital) / instrument risk %
- target risk = 12%

Closing rule
- Trailing stop
- Stop loss gap = 0.5 x instrument risk % x closing price at entry


### AUDUSD

In [1]:
import fxcmpy

In [2]:
TOKEN = 'FXCM TOKEN'
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', server='demo', log_file='log.txt')

In [5]:
con.is_connected()

True

In [30]:
audusd = con.get_candles('AUD/USD', period='D1', number=50)
audusd.drop(audusd.tail(1).index, inplace=True)
audusd.tail()

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2020-08-07 21:00:00,0.72342,0.71564,0.72423,0.71438,0.72365,0.71608,0.72441,0.71454,276627
2020-08-09 21:00:00,0.71555,0.71570,0.71593,0.71555,0.71633,0.71659,0.71659,0.71633,14
2020-08-10 21:00:00,0.71570,0.71473,0.71833,0.71395,0.71659,0.71518,0.71848,0.71411,171198
2020-08-11 21:00:00,0.71473,0.71391,0.71888,0.71341,0.71518,0.71485,0.71905,0.71359,234654
2020-08-12 21:00:00,0.71391,0.71606,0.71751,0.71084,0.71485,0.71639,0.71769,0.71097,246866


In [37]:
ins_risk = volatility(audusd['bidclose'], 25) * 16
ins_risk

0.08869172370286511

In [35]:
"""
Formula 21: Minimum capital
Minimum capital = (minimum exposure x instrument risk %) / target risk %
"""
target_risk = 0.12
min_exposure = audusd['askclose'].iloc[-1] * 1000
min_capital = min_exposure * ins_risk / target_risk
print(min_capital)

529.4821995291295


## Position Sizing

### Notional exposure = (target risk x capital) / instrument risk %

In [38]:
capital = 2282
expose = target_risk * capital / ins_risk
print(expose)

3087.548517124533


## Instrument risk calculation

In [ ]:
ASII['pct'] = ASII['close'].pct_change()
ASII.tail()

,open,high,low,close,volume,pct
date,,,,,,
2020-07-24,5175.0,5175.0,5000.0,5050.0,32338100,-0.024155
2020-07-27,5075.0,5150.0,5025.0,5125.0,17645200,0.014851
2020-07-28,5175.0,5225.0,5100.0,5175.0,30210400,0.009756
2020-07-29,5175.0,5225.0,5050.0,5075.0,32010100,-0.019324
2020-07-30,5025.0,5225.0,5000.0,5150.0,32720500,0.014778


In [ ]:
vola_window = 25
ASII['std'] = ASII['pct'].rolling(vola_window).std()
ASII.tail()

,open,high,low,close,volume,pct,std
date,,,,,,,
2020-07-24,5175.0,5175.0,5000.0,5050.0,32338100,-0.024155,0.019233
2020-07-27,5075.0,5150.0,5025.0,5125.0,17645200,0.014851,0.019414
2020-07-28,5175.0,5225.0,5100.0,5175.0,30210400,0.009756,0.019475
2020-07-29,5175.0,5225.0,5050.0,5075.0,32010100,-0.019324,0.019562
2020-07-30,5025.0,5225.0,5000.0,5150.0,32720500,0.014778,0.019258


In [ ]:
# Instrument Risk
ASII['ins_risk'] = ASII['std'] * 16
ASII.tail()

,open,high,low,close,volume,pct,std,ins_risk
date,,,,,,,,
2020-07-24,5175.0,5175.0,5000.0,5050.0,32338100,-0.024155,0.019233,0.307725
2020-07-27,5075.0,5150.0,5025.0,5125.0,17645200,0.014851,0.019414,0.310627
2020-07-28,5175.0,5225.0,5100.0,5175.0,30210400,0.009756,0.019475,0.311602
2020-07-29,5175.0,5225.0,5050.0,5075.0,32010100,-0.019324,0.019562,0.313000
2020-07-30,5025.0,5225.0,5000.0,5150.0,32720500,0.014778,0.019258,0.308122


## Simple Moving Average calculations

In [ ]:
fast_period = 16
slow_period = 64
ASII['fast_ma'] = ASII['close'].rolling(fast_period).mean()
ASII['slow_ma'] = ASII['close'].rolling(slow_period).mean()
ASII['ma_cross'] = ASII['fast_ma'] - ASII ['slow_ma']
ASII.tail()

,open,high,low,close,volume,pct,std,ins_risk,fast_ma,slow_ma,ma_cross
date,,,,,,,,,,,
2020-07-24,5175.0,5175.0,5000.0,5050.0,32338100,-0.024155,0.019233,0.307725,5038.1250,4517.421875,520.703125
2020-07-27,5075.0,5150.0,5025.0,5125.0,17645200,0.014851,0.019414,0.310627,5055.3125,4539.375000,515.937500
2020-07-28,5175.0,5225.0,5100.0,5175.0,30210400,0.009756,0.019475,0.311602,5076.2500,4563.359375,512.890625
2020-07-29,5175.0,5225.0,5050.0,5075.0,32010100,-0.019324,0.019562,0.313000,5091.5625,4585.781250,505.781250
2020-07-30,5025.0,5225.0,5000.0,5150.0,32720500,0.014778,0.019258,0.308122,5106.5625,4607.500000,499.062500


## Position Sizing

### Notional exposure = (target risk x capital) / instrument risk %

In [ ]:
risk_target = 0.12
capital = 20000000
expose = risk_target * capital / ASII['ins_risk'].iloc[-1]
expose

7789129.230926564